In [2]:
# imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib


import warnings
warnings.filterwarnings('ignore')

In [3]:
list_nPCs = [10, 15, 20, 25, 30]
takeprofit = 0.12
stoploss = 0.05

In [4]:
# (a) Load previously built datasets : we just need train sets and validation sets here
trainset_final = pd.read_csv('./Data/Trainset_final_SL{}_TP{}_btc.csv'.format(stoploss, takeprofit))
trainset = pd.read_csv('./Data/Trainset_SL{}_TP{}_btc.csv'.format(stoploss, takeprofit))

validation_set_final = pd.read_csv('./Data/Validationset_final_SL{}_TP{}_btc.csv'.format(stoploss, takeprofit))
validation_set = pd.read_csv('./Data/Validationset_SL{}_TP{}_btc.csv'.format(stoploss, takeprofit))

In [5]:
for nPCs in list_nPCs:
    print(nPCs)
    X = trainset_final.iloc[:, :nPCs]
    y = trainset["result"]

    # Build model and train it
    classifier = Sequential()

    dropout_fraction = 0.2

    #First Hidden Layer
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=nPCs))
    classifier.add(Dropout(dropout_fraction))

    #Second, third and fourth  hidden Layers
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))

    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))

    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dropout(dropout_fraction))

    #Output Layer
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    #Compiling the neural network
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    #Fitting the data to the training dataset
    classifier.fit(X,y, batch_size=500, epochs=75, verbose =1)

    pk.dump(classifier, open("./Models/DL_model_{}PC_SL{}_TP{}_btc.pkl".format(nPCs, stoploss, takeprofit), 'wb'))

10
Epoch 1/75
1/1 [==============================] - 1s 826ms/step - loss: 0.6931 - accuracy: 0.6393
Epoch 2/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6928 - accuracy: 0.7164
Epoch 3/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6925 - accuracy: 0.7164
Epoch 4/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6922 - accuracy: 0.7164
Epoch 5/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6919 - accuracy: 0.7164
Epoch 6/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6916 - accuracy: 0.7164
Epoch 7/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6913 - accuracy: 0.7164
Epoch 8/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6909 - accuracy: 0.7164
Epoch 9/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6906 - accuracy: 0.7164
Epoch 10/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6902 - accuracy: 0.7164
Epoch 11/75
1/1 [=================

TypeError: can't pickle weakref objects

In [6]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2

In [ ]:
# # (b) Build and train several models different amounts of PCs
# for nPCs in list_nPCs:
#     print(nPCs)
#     X = trainset_final.iloc[:, :nPCs]
#     y = trainset["result"]

#     # Build model and train it
#     classifier = Sequential()
#     #First Hidden Layer
#     classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=nPCs))
#     #Second, third and fourth  hidden Layers
#     classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
#     classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#     classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

#     #Output Layer
#     classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#     #Compiling the neural network
#     classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
#     #Fitting the data to the training dataset
#     classifier.fit(X,y, batch_size=500, epochs=75, verbose =1)

#     # with open("./Models/DL_model_{}PC_SL{}_TP{}_btc.pkl".format(nPCs, stoploss, takeprofit), 'wb') as f:
#     #     pk.dump(classifier, f)
#     pk.dump(classifier, open("./Models/DL_model_{}PC_SL{}_TP{}_btc.pkl".format(nPCs, stoploss, takeprofit), 'wb'))

In [ ]:
accuracies, nPCs_list = [], []
for nPCs in list_nPCs:
    print(nPCs)
    with open("./Models/DL_model_{}PC_SL{}_TP{}_btc.pkl".format(nPCs, stoploss, takeprofit), 'rb') as f:
        clf = pk.load(f)
    # Compute predictions on testset
    preds = (clf.predict(validation_set_final.iloc[:, :nPCs]) > 0.5)*1

    # Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
    validation_set1 = validation_set[preds == 1].copy()
    accuracies.append(np.mean(preds == list(validation_set1['result'])))
    nPCs_list.append(nPCs)

recap = pd.DataFrame({'nPCs' : list(nPCs_list), 'Accuracy' : (list(accuracies))})
recap.to_csv('./Results/Comparative_All_models_SL{}_TP{}_btc.csv'.format(stoploss, takeprofit), index = False)
print(recap)